In [9]:
from alphagenome.data import genome
from alphagenome.models import dna_client, variant_scorers
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()
ALPHAGENOME_KEY=os.getenv('ALPHAGENOME_KEY')
save_predictions = True

In [10]:
# Load tsv file which only has variants above -log10p of 7.
df = pd.read_csv('gwas1_log10p_gt_7.tsv', sep='\t')
df.CHROM = 'chr' + df.CHROM.astype(str)
df

,CHROM,GENPOS,ID,ALLELE0,ALLELE1,A1FREQ,A1FREQ_CASES,A1FREQ_CONTROLS,N,N_CASES,N_CONTROLS,TEST,BETA,SE,CHISQ,LOG10P,EXTRA
0,chr6,26205065,6:26205065:G:A,G,A,0.468846,0.490179,0.496948,275488,15579,259909,ADD,0.066559,0.012493,28.3848,7.00242,NaN
1,chr6,26210918,6:26210918:G:T,G,T,0.273908,0.289524,0.290326,275488,15579,259909,ADD,0.075221,0.013859,29.4583,7.24311,NaN
2,chr6,26214245,6:26214245:T:C,T,C,0.474028,0.495378,0.502440,275488,15579,259909,ADD,0.066576,0.012478,28.4681,7.02110,NaN
3,chr6,26225804,6:26225804:C:G,C,G,0.284913,0.301078,0.301990,275488,15579,259909,ADD,0.073885,0.013698,29.0931,7.16125,NaN
4,chr6,26233159,6:26233159:A:G,A,G,0.274226,0.291707,0.290663,275488,15579,259909,ADD,0.079009,0.013844,32.5706,7.93952,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,chr20,49274119,20:49274119:A:G,A,G,0.391124,0.373098,0.414567,275488,15579,259909,ADD,-0.077317,0.013166,34.4866,8.36734,NaN
338,chr20,49275101,20:49275101:C:T,C,T,0.390888,0.372745,0.414317,275488,15579,259909,ADD,-0.077830,0.013168,34.9351,8.46739,NaN
339,chr20,49278429,20:49278429:G:A,G,A,0.389233,0.371333,0.412563,275488,15579,259909,ADD,-0.077548,0.013177,34.6343,8.40029,NaN
340,chr20,49291253,20:49291253:A:G,A,G,0.390944,0.372713,0.414377,275488,15579,259909,ADD,-0.077937,0.013169,35.0244,8.48732,NaN


In [11]:
dna_model = dna_client.create(ALPHAGENOME_KEY)

In [4]:
# Sequence length and scorer selection

sequence_length = '1MB'  # "16KB", "100KB", "500KB", "1MB"
sequence_length = dna_client.SUPPORTED_SEQUENCE_LENGTHS[
    f'SEQUENCE_LENGTH_{sequence_length}'
]

scorer_selections = {
    'rna_seq': True,
    'cage': False,
    'procap': False,
    'atac': False,
    'dnase': False,
    'chip_histone': False,
    'chip_tf': False,
    'polyadenylation': False,
    'splice_sites': False,
    'splice_site_usage': False,
    'splice_junctions': False,
}

all_scorers = variant_scorers.RECOMMENDED_VARIANT_SCORERS
selected_scorers = [
    all_scorers[key]
    for key in all_scorers
    if scorer_selections.get(key.lower(), False)
]

In [5]:
results = []
for i, row in tqdm(df.iterrows(), total=len(df)):
  variant = genome.Variant(
      chromosome=str(row.CHROM),
      position=int(row.GENPOS),
      reference_bases=row.ALLELE0,
      alternate_bases=row.ALLELE1,
      name=row.ID,
  )
  interval = variant.reference_interval.resize(sequence_length)

  variant_scores = dna_model.score_variant(
      interval=interval,
      variant=variant,
      variant_scorers=selected_scorers,
      organism=dna_client.Organism.HOMO_SAPIENS,
  )
  results.append(variant_scores)

100%|█████████████████████████████████████████| 342/342 [06:31<00:00,  1.14s/it]


In [14]:
df_scores = variant_scorers.tidy_scores(results)
df_scores['abs_quantile_score'] = abs(df_scores['quantile_score'])
df_scores = df_scores.sort_values('abs_quantile_score', ascending=False)

df_scores.to_csv('variant-scores_rnaseq_decodeme_significant_all-tissues.csv.gz', index=False, compression='gzip')

df_scores

,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,biosample_name,biosample_type,biosample_life_stage,gtex_tissue,data_source,endedness,genetically_modified,raw_score,quantile_score,abs_quantile_score
2095528,chr20:48905549:CT>C,chr20:48381261-49429837:.,ENSG00000230758,SNAP23P1,processed_pseudogene,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,endothelial cell,in_vitro_differentiated_cells,adult,,encode,single,False,1.833034e-01,9.999800e-01,9.999800e-01
2609855,chr20:49012496:T>TTTTG,chr20:48488208-49536784:.,ENSG00000226284,ARPC3P1,processed_pseudogene,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,MG63,cell_line,child,,encode,paired,False,1.973138e-01,9.999800e-01,9.999800e-01
3102464,chr20:49105400:TTTG>T,chr20:48581113-49629689:.,ENSG00000124212,PTGIS,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,endothelial cell of coronary artery,primary_cell,adult,,encode,paired,False,-3.197962e-01,-9.999800e-01,9.999800e-01
3102480,chr20:49105400:TTTG>T,chr20:48581113-49629689:.,ENSG00000124212,PTGIS,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,dermis microvascular lymphatic vessel endothel...,primary_cell,adult,,encode,paired,False,-4.071872e-01,-9.999800e-01,9.999800e-01
3102472,chr20:49105400:TTTG>T,chr20:48581113-49629689:.,ENSG00000124212,PTGIS,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,bladder microvascular endothelial cell,primary_cell,adult,,encode,paired,False,-4.263840e-01,-9.999800e-01,9.999800e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87802,chr6:26214245:T>C,chr6:25689957-26738533:.,ENSG00000287050,ENSG00000287050,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,body of pancreas,tissue,adult,,encode,paired,False,0.000000e+00,1.137757e-12,1.137757e-12
973938,chr6:26306054:T>C,chr6:25781766-26830342:.,ENSG00000196866,H2AC7,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,bronchial smooth muscle cell,primary_cell,adult,,encode,paired,False,-2.384186e-07,1.137757e-12,1.137757e-12
1208546,chr6:26325805:C>T,chr6:25801517-26850093:.,ENSG00000287050,ENSG00000287050,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,Ammon's horn,tissue,adult,Brain_Hippocampus,gtex,paired,False,-4.768372e-07,1.137757e-12,1.137757e-12
2117894,chr20:48906783:C>T,chr20:48382495-49431071:.,ENSG00000236874,ENSG00000236874,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,urinary bladder,tissue,embryonic,,encode,paired,False,-4.768372e-07,1.137757e-12,1.137757e-12


In [17]:
ontologies = ['UBERON:0000955']
gene_types = ['protein_coding']
df_scores_subset = df_scores[df_scores['ontology_curie'].isin(ontologies) & df_scores['gene_type'].isin(gene_types)]

if save_predictions:
  df_scores_subset.to_csv('variant-scores_rnaseq_decodeme_significant_brain_protein-coding.csv.gz', index=False, compression='gzip')

df_scores_subset

,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,biosample_name,biosample_type,biosample_life_stage,gtex_tissue,data_source,endedness,genetically_modified,raw_score,quantile_score,abs_quantile_score
3146736,chr20:49109692:C>CTTTT,chr20:48585404-49633980:.,ENSG00000124212,PTGIS,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,1.050655e-01,9.999366e-01,9.999366e-01
1284171,chr6:26328470:A>AT,chr6:25804182-26852758:.,ENSG00000124557,BTN1A1,protein_coding,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,-9.972215e-02,-9.999220e-01,9.999220e-01
3283438,chr20:49132759:CCAAAA>C,chr20:48608473-49657049:.,ENSG00000124228,DDX27,protein_coding,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,7.458559e-02,9.999164e-01,9.999164e-01
3103176,chr20:49105400:TTTG>T,chr20:48581113-49629689:.,ENSG00000124212,PTGIS,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,-8.859038e-02,-9.998872e-01,9.998872e-01
3460368,chr20:49153402:T>TA,chr20:48629114-49677690:.,ENSG00000124212,PTGIS,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,7.188678e-02,9.998845e-01,9.998845e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3016056,chr20:49100868:T>C,chr20:48576580-49625156:.,ENSG00000124212,PTGIS,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,-2.980232e-06,-3.459415e-02,3.459415e-02
2494273,chr20:48955014:C>G,chr20:48430726-49479302:.,ENSG00000124201,ZNFX1,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,-2.801418e-06,-3.459415e-02,3.459415e-02
1040886,chr6:26317156:C>T,chr6:25792868-26841444:.,ENSG00000124564,SLC17A3,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,-1.430511e-06,-2.306788e-02,2.306788e-02
882492,chr6:26302950:T>G,chr6:25778662-26827238:.,ENSG00000181315,ZNF322,protein_coding,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,brain,tissue,adult,,encode,paired,False,-8.344650e-07,-1.153548e-02,1.153548e-02
